In [5]:
import asyncio

# Função assíncrona que será chamada dentro de 'minha_funcao'
async def outra_funcao(i):
    await asyncio.sleep(0.5)  # Simula um tempo de espera
    print(f"Outra tarefa {i} concluída")

# Função principal que será chamada nas tarefas
async def minha_funcao(i, sem):
    async with sem:
        await outra_funcao(i)  # Chama a outra função assíncrona
        await asyncio.sleep(1)
        print(f"Tarefa {i} concluída")

# Função que organiza a execução das tarefas
async def main():
    lista = list(range(10))  # Sua lista de itens para iterar
    sem = asyncio.Semaphore(10)  # Limita a 10 tarefas simultâneas

    # Criar e executar as tarefas em paralelo
    tarefas = [asyncio.create_task(minha_funcao(i, sem)) for i in lista]
    await asyncio.gather(*tarefas)

# No Jupyter Notebook ou ambiente interativo, basta usar `await` para rodar a função principal
await main()


Outra tarefa 0 concluída
Outra tarefa 2 concluída
Outra tarefa 6 concluída
Outra tarefa 9 concluída
Outra tarefa 8 concluída
Outra tarefa 5 concluída
Outra tarefa 7 concluída
Outra tarefa 4 concluída
Outra tarefa 1 concluída
Outra tarefa 3 concluída
Tarefa 0 concluída
Tarefa 6 concluída
Tarefa 7 concluída
Tarefa 3 concluída
Tarefa 1 concluída
Tarefa 5 concluída
Tarefa 4 concluída
Tarefa 8 concluída
Tarefa 2 concluída
Tarefa 9 concluída


In [6]:
import asyncio
import pandas as pd

# Criando um DataFrame de exemplo
df = pd.DataFrame({
    "id": range(10), 
    "valor": [x * 2 for x in range(10)]
})

# Função assíncrona que será chamada dentro de 'minha_funcao'
async def outra_funcao(linha):
    await asyncio.sleep(0.5)  # Simula um tempo de espera
    print(f"Outra tarefa {linha['id']} concluída com valor {linha['valor']}")

# Função principal que será chamada nas tarefas
async def minha_funcao(linha, sem):
    async with sem:
        await outra_funcao(linha)  # Chama a outra função assíncrona
        await asyncio.sleep(1)
        print(f"Tarefa {linha['id']} concluída")

# Função que organiza a execução das tarefas
async def main():
    sem = asyncio.Semaphore(5)  # Limita a 5 tarefas simultâneas

    # Criar e executar as tarefas em paralelo iterando sobre o DataFrame
    tarefas = [asyncio.create_task(minha_funcao(linha, sem)) for _, linha in df.iterrows()]
    await asyncio.gather(*tarefas)

# 🔹 Se já estivermos dentro de um loop de eventos, usamos `await`
try:
    await main()  # Para Jupyter Notebook ou outros loops ativos
except RuntimeError:
    asyncio.run(main())  # Para execução normal em scripts Python


Outra tarefa 0 concluída com valor 0
Outra tarefa 2 concluída com valor 4
Outra tarefa 4 concluída com valor 8
Outra tarefa 1 concluída com valor 2
Outra tarefa 3 concluída com valor 6
Tarefa 0 concluída
Tarefa 4 concluída
Tarefa 3 concluída
Tarefa 2 concluída
Tarefa 1 concluída
Outra tarefa 5 concluída com valor 10
Outra tarefa 7 concluída com valor 14
Outra tarefa 9 concluída com valor 18
Outra tarefa 6 concluída com valor 12
Outra tarefa 8 concluída com valor 16
Tarefa 5 concluída
Tarefa 9 concluída
Tarefa 8 concluída
Tarefa 7 concluída
Tarefa 6 concluída


In [8]:
import snowflake.snowpark as snowpark
from snowflake.snowpark import Session
from dotenv import load_dotenv
from pyspark.sql.functions import col
import asyncio
import os

load_dotenv()


connection_parameters = {
   "account": os.getenv('account_snow'),
   "user": os.getenv('user_snow'),
   "password":os.getenv('password_snow'),
   "role": "ACCOUNTADMIN",
   "database": "NEW_DB", 
   "schema": "PUBLIC"
  
}  

session = Session.builder.configs(connection_parameters).create()

In [9]:
df= session.create_dataframe([["a", 3], ["b", 4], ["c", 5]], schema=["id", "valor"])

In [12]:
df.show()

------------------
|"ID"  |"VALOR"  |
------------------
|a     |3        |
|b     |4        |
|c     |5        |
------------------



In [13]:


# 🔹 Função assíncrona que será chamada dentro de 'minha_funcao'
async def outra_funcao(id_valor, valor):
    await asyncio.sleep(0.5)  # Simula um tempo de espera
    print(f"Outra tarefa {id_valor} concluída com valor {valor}")

# 🔹 Função principal que será chamada nas tarefas
async def minha_funcao(id_valor, valor, sem):
    async with sem:
        await outra_funcao(id_valor, valor)  # Chama a outra função assíncrona
        await asyncio.sleep(1)
        print(f"Tarefa {id_valor} concluída")

# 🔹 Função que organiza a execução das tarefas
async def main():
    sem = asyncio.Semaphore(5)  # Limita a 5 tarefas simultâneas
    
    # 🔹 Criar tarefas iterando diretamente sobre as linhas do Snowspark DataFrame
    tarefas = [asyncio.create_task(minha_funcao(row["ID"], row["VALOR"], sem)) for row in df.collect()]
    
    await asyncio.gather(*tarefas)

# 🔹 Executar a função principal corretamente em qualquer ambiente
try:
    await main()  # Para Jupyter Notebook ou outros loops ativos
except RuntimeError:
    asyncio.run(main())  # Para execução normal em scripts Python


Outra tarefa a concluída com valor 3
Outra tarefa b concluída com valor 4
Outra tarefa c concluída com valor 5
Tarefa a concluída
Tarefa b concluída
Tarefa c concluída
